<a href="https://colab.research.google.com/github/pitthexai/ICHI2023_EBAIC/blob/main/EBAIC2024_Workshop/Track02_Medical-Imaging-Informatics/EBAIC2024_KneeSegmentationBiasEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bony Anatomy Segmentation

In [1]:
!pip install segmentation-models-pytorch albumentations torchmetrics pydicom nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import copy

from zipfile import ZipFile

from io import BytesIO
from gzip import GzipFile

import os

import cv2
from PIL import Image

import pydicom
import nibabel
from nibabel import FileHolder, Nifti1Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils as smp_utils

from torchmetrics.classification import MulticlassJaccardIndex, Dice
import torchvision.transforms.functional as TF

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
RANDOM_STATE = 42

## Data Preprocessing

In [11]:
# Directories for

filename = "knee_seg_sample.csv"
directory = "KneeSample"
zipfile = "knee_sample.zip"

classes = {
    0: "Background",
    1: "R Patella",
    2: "R Femur",
    3: "R Tibia",
    4: "R Fibula",
    5: "L Patella",
    6: "L Femur",
    7: "L Tibia",
    8: "L Fibula"
}

zipfile_loc = f"/content/drive/MyDrive/GoogleColabProjects/EBAIC_2024/KneeDataset/{zipfile}"
data_location = f"/content/data/{directory}/{filename}"
data_save_location = f"/content/data/{directory}/"

In [12]:
if not os.path.exists(data_save_location):
    with ZipFile(zipfile_loc, 'r') as zipf:
        zipf.extractall("/content/")

In [13]:
def generate_train_test_split(csv_pth, filter_query=None):
    data_records = pd.read_csv(csv_pth)
    data_records = data_records[data_records.id != 9025994].reset_index(drop=True)
    if filter_query:
        data_records = data_records.query(filter_query)

    train, test = train_test_split(data_records.id.unique(), test_size=0.3, random_state=42)
    valid, test = train_test_split(test, test_size=0.5, random_state=42)

    train = data_records[data_records.id.isin(train)].reset_index(drop=True)
    valid = data_records[data_records.id.isin(valid)].reset_index(drop=True)
    test = data_records[data_records.id.isin(test)].reset_index(drop=True)

    return train, valid, test

In [14]:
def balance_dataset(data, by_filter1, by_filter2):
    filtered1 = data.query(by_filter1)
    filtered2 = data.query(by_filter2)

    min_sample_size = np.minimum(len(filtered1), len(filtered2))
    samp1 = filtered1.sample(min_sample_size,  random_state = 42)
    samp2 = filtered2.sample(min_sample_size,  random_state = 42)

    balanced_data = pd.concat([samp1, samp2]).reset_index(drop=True)

    print(f"Training dataset reduced from size of {len(data)} samples to a balanced dataset of size {len(balanced_data)} samples")

    return balanced_data

In [15]:
## Baseline Datasets
train_all, valid_all, test_all = generate_train_test_split(data_location)
train_white, valid_white, test_white = generate_train_test_split(data_location, filter_query="P02RACE == '1: White or Caucasian'")
train_black, valid_black, test_black = generate_train_test_split(data_location, filter_query="P02RACE == '2: Black or African American'")
train_male, valid_male, test_male = generate_train_test_split(data_location, filter_query="P02SEX == '1: Male'")
train_female, valid_female, test_female = generate_train_test_split(data_location, filter_query="P02SEX == '2: Female'")

balanced_gender_train = balance_dataset(train_all, "P02SEX == '1: Male'", "P02SEX == '2: Female'")
balanced_race_train = balance_dataset(train_all, "P02RACE == '1: White or Caucasian'", "P02RACE == '2: Black or African American'")

Training dataset reduced from size of 70 samples to a balanced dataset of size 60 samples
Training dataset reduced from size of 70 samples to a balanced dataset of size 62 samples


## Baseline Models


In [16]:
class BonyAnatomyJointSegmentationDataset(Dataset):
    def __init__(self, root_dir, ids, num_classes, transforms=None, preprocessing=None):
        self.root_dir = root_dir
        self.pids = ids
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def load_dicom(self, path):
        dicom_img = pydicom.dcmread(path)
        return dicom_img.pixel_array.astype(np.float32)

    def load_nii(self, path):
        nii_annot = nibabel.load(path)
        nii_annot_data = nii_annot.get_fdata()

        if len(nii_annot_data.shape) == 3 and nii_annot_data.shape[-1] > 1:
            if nii_annot_data.shape[-1] == 2:
                nii_annot_data = nii_annot_data[:, :, 1]
            else:
                nii_annot_data = nii_annot_data[:, :, nii_annot_data.shape[-1]//2]

            nii_annot_data = np.expand_dims(nii_annot_data, axis=-1)


        nii_annot_data = cv2.rotate(nii_annot_data, cv2.ROTATE_90_CLOCKWISE)
        nii_annot_data = cv2.flip(nii_annot_data, 1)
        return nii_annot_data

    def get_file_path(self, filename):
        return os.path.join(self.root_dir, filename)

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):

        image = self.load_dicom(self.get_file_path(os.path.join("Images/", str(self.pids[idx]) + ".dcm")))
        mask = self.load_nii(self.get_file_path(os.path.join("Annotations", str(self.pids[idx]) + ".nii.gz")))

#         if len(np.unique(mask)) != self.num_classes:
#             print(self.pids[idx])
        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]

        return image.type(torch.FloatTensor), mask.long()


In [17]:
def train_model(train_loader, valid_loader, num_classes=9):
    encoder = "resnet18"
    encoder_weights = "imagenet"
    activation = None


    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = copy.deepcopy(smp.Unet(encoder_name=encoder, encoder_weights=encoder_weights, in_channels=1,
                    classes=num_classes, activation=activation)).to(device)
    model.encoder.requires_grad_ = False
    model.decoder.requires_grad_ = False

    loss = nn.CrossEntropyLoss()
    loss.__name__=" loss"

    multi_jaccard = MulticlassJaccardIndex(num_classes=num_classes, average="macro").to(device)
    multi_jaccard.__name__ = "iou_score"
    metrics = [multi_jaccard]

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-04)

    # create epoch runners
    # it is a simple loop of iterating over dataloader`s samples
    train_epoch = smp.utils.train.TrainEpoch(
        model,
        loss=loss,
        metrics=metrics,
        optimizer=optimizer,
        device=device,
        verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
        model,
        loss=loss,
        metrics=metrics,
        device=device,
        verbose=True,
    )

    max_score = 0

    for i in range(1, 51):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        # do something (save model, change lr, etc.)
        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            torch.save(model, './best_model.pth')
            print('Model saved!')

    # Return best model
    model = torch.load('./best_model.pth')
    return model

In [18]:
def test_model(model, test_loader, class_labels):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"

    fig, ax = plt.subplots(nrows=5, ncols=3, figsize=(10, 10), sharex=True, sharey=True)

    x, y = next(iter(test_loader))
    out = torch.softmax(model(x.to(device)), dim=1)
    #out = out.detach().cpu().numpy()

    for i, pred in enumerate(out):
        uni_channels = torch.argmax(pred, dim=0).unique()
        pred = pred.detach().cpu().numpy()
        ax[i][0].imshow(x[i].squeeze(), cmap="gray")
        yi = y[i].squeeze()
        ax[i][1].imshow(yi)

        # Merge predicted masks into one image
        mask = np.where(pred[uni_channels[0].item(),:,:] > 0.5, uni_channels[0].item(), 0)
        for channel in uni_channels[1:]:
            channel = channel.item()
            channel_mask = np.where(pred[channel,:,:] > 0.5, channel, 0)
            mask = mask | channel_mask
        ax[i][2].imshow(mask)

    ax[0][0].set_title("Image")
    ax[0][1].set_title("Ground Truth Mask")
    ax[0][2].set_title("Predicted Mask")
    plt.xticks([])
    plt.yticks([])

    plt.show()

    multi_jaccard = MulticlassJaccardIndex(num_classes=num_classes, average="none").cuda()
    multi_jaccard.__name__ = "iou"
    metrics = [multi_jaccard]

    results = torch.zeros((1, num_classes))
    for x, y in test_loader:
        results += multi_jaccard(torch.softmax(model(x.cuda()), dim=1), y.cuda()).detach().cpu()

    results = results/len(test_loader)

    for i in range(0, len(class_labels)):
        print(f"{class_labels[i]} (Class {i}): {results[0][i]}")

    print(f"Mean Testing IoU: {multi_jaccard(torch.softmax(model(x.cuda()), dim=1), y.cuda()).mean()}")
    return results

In [21]:
augmentations = A.Compose([A.Resize(256, 256), ToTensorV2()])

num_classes = 9

In [22]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_all.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_all.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location,test_all.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [23]:
baseline_model = train_model(train_loader, valid_loader, num_classes)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 382MB/s]



Epoch: 1
train:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


valid: 100%|██████████| 1/1 [00:02<00:00,  2.60s/it,  loss - 1.937, iou_score - 0.04938]
Model saved!

Epoch: 2
train:  40%|████      | 2/5 [00:05<00:07,  2.45s/it,  loss - 1.482, iou_score - 0.1642]

/usr/local/lib/python3.10/dist-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


...

/usr/local/lib/python3.10/dist-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


valid: 100%|██████████| 1/1 [00:03<00:00,  3.07s/it,  loss - 0.7185, iou_score - 0.4141]
Model saved!

Epoch: 6
train:  40%|████      | 2/5 [00:05<00:06,  2.23s/it,  loss - 0.6407, iou_score - 0.5209]

/usr/local/lib/python3.10/dist-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)


valid: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it,  loss - 0.6119, iou_score - 0.5288]
Model saved!

Epoch: 7
train:  40%|████      | 2/5 [00:08<00:13,  4.42s/it,  loss - 0.545, iou_score - 0.5499]


KeyboardInterrupt: 

In [ ]:
baseline_results = test_model(baseline_model, test_loader, classes)

## Gender Specific Baselines



### Male

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_male.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_male.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location, test_male.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
baseline_male_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
baseline_male_results = test_model(baseline_male_model, test_loader, classes)

### Female

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_female.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_female.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location, test_female.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
baseline_female_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
baseline_female_results = test_model(baseline_female_model, test_loader, classes)

## Race Specific Baselines

### White/Caucasian

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_white.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_white.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location, test_white.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
baseline_white_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
baseline_white_results = test_model(baseline_white_model, test_loader, classes)

### Black/African American (AA)

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_black.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_black.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location, test_black.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
baseline_black_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
baseline_black_results = test_model(baseline_black_model, test_loader, classes)

## Balanced Models

### Gender

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, balanced_gender_train.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_all.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location, test_all.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
balanced_gender_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
balanced_gender_results = test_model(balanced_gender_model, test_loader, classes)

### Race

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, balanced_race_train.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_all.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location, test_all.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
balanced_race_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
balanced_race_results = test_model(balanced_race_model, test_loader, classes)

## Stratified Models

In [ ]:
from sklearn.model_selection import StratifiedKFold
class StratifiedSampler:
    """
    Based on this Pytorch discussion board post
    https://discuss.pytorch.org/t/how-to-enable-the-dataloader-to-sample-from-each-class-with-equal-probability/911/6
    """
    def __init__(self, stratify_on, batch_size):
        self.stratify_on = stratify_on
        self.batch_size = batch_size
        self.nsplits = int(len(stratify_on) / batch_size)

    def gen_stratified_sample(self):
        s = StratifiedKFold(n_splits = self.nsplits)

        X = np.arange(0, len(self.stratify_on))
        s.get_n_splits(X, self.stratify_on)
        for train_idx, valid_idx in s.split(X, self.stratify_on):
            yield valid_idx

    def __iter__(self):
        return iter(self.gen_stratified_sample())

    def __len__(self):
        return self.nsplits


### Gender

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_all.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_all.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location,test_all.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, num_workers=2, batch_sampler=StratifiedSampler(train_all.P02SEX, batch_size=16))
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
stratified_gender_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
stratified_gender_results = test_model(stratified_gender_model, test_loader, classes)

### Race

In [ ]:
train_set = BonyAnatomyJointSegmentationDataset(data_save_location, train_all.id, num_classes,
                                                transforms=augmentations)

valid_set = BonyAnatomyJointSegmentationDataset(data_save_location, valid_all.id, num_classes,
                                                transforms=augmentations)

test_set = BonyAnatomyJointSegmentationDataset(data_save_location,test_all.id, num_classes,
                                               transforms=augmentations)

train_loader = DataLoader(train_set, num_workers=2, batch_sampler=StratifiedSampler(train_all.P02RACE, batch_size=16))
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=5, shuffle=False, num_workers=2)

In [ ]:
stratified_race_model = train_model(train_loader, valid_loader, num_classes)

In [ ]:
stratified_race_results = test_model(stratified_race_model, test_loader, classes)

## Evaluation Bias

In [ ]:
def eval_bias(model, dataset, metric, num_classes = 9):
    results = torch.zeros((1, num_classes))
    test_loader = generate_dataloader(dataset, num_classes)
    for x, y in test_loader:
        results += metric(torch.softmax(model(x.cuda()), dim=1), y.cuda()).detach().cpu()

    return results/len(test_loader)

def generate_dataloader(dataset, num_classes):
    augmentations = A.Compose([A.Resize(256, 256), ToTensorV2()]) # A.OneOf([A.Emboss(), Canny()
    test_set = BonyAnatomyJointSegmentationDataset(data_save_location, dataset.id, num_classes,
                                                   transforms=augmentations)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=2)

    return test_loader

In [ ]:
multi_jaccard = MulticlassJaccardIndex(num_classes=9, average="none").cuda()
multi_jaccard.__name__ = "iou"

In [ ]:
data_records = pd.read_csv(data_location)

In [ ]:
male_group = data_records[data_records.P02SEX == "1: Male"].reset_index(drop=True)
female_group = data_records[data_records.P02SEX == "2: Female"].reset_index(drop=True)
white_caucasian_group = data_records[data_records.P02RACE == "1: White or Caucasian"].reset_index(drop=True)
black_aa_group = data_records[data_records.P02RACE == "2: Black or African American"].reset_index(drop=True)

In [ ]:
gender_male_metrics_baseline = eval_bias(baseline_model, male_group, multi_jaccard)
gender_female_metrics_baseline = eval_bias(baseline_model, female_group, multi_jaccard)
race_white_metrics_baseline = eval_bias(baseline_model, white_caucasian_group, multi_jaccard)
race_black_metrics_baseline = eval_bias(baseline_model, black_aa_group, multi_jaccard)

base_dict = {
    "Gender: Male": gender_male_metrics_baseline.numpy().squeeze(),
    "Gender: Female": gender_female_metrics_baseline.numpy().squeeze(),
    "Race: White/Caucasian": race_white_metrics_baseline.numpy().squeeze(),
    "Race: Black/AA": race_black_metrics_baseline.numpy().squeeze(),
}

baseline_res = pd.DataFrame(base_dict).T
baseline_res["Average"] = baseline_res.mean(axis=1)
baseline_res.columns = [val for key, val in classes.items()] + ["Average"]
baseline_res

In [ ]:
gender_male_metrics_balanced = eval_bias(balanced_gender_model, male_group, multi_jaccard)
gender_female_metrics_balanced = eval_bias(balanced_gender_model, female_group, multi_jaccard)

balanced_gender = {
    "Gender: Male": gender_male_metrics_balanced.numpy().squeeze(),
    "Gender: Female": gender_female_metrics_balanced.numpy().squeeze(),
}
balanced_gender_res = pd.DataFrame(balanced_gender).T
balanced_gender_res["Average"] =  balanced_gender_res.mean(axis=1)
balanced_gender_res.columns = [val for key, val in classes.items()] + ["Average"]

balanced_gender_res

In [ ]:
race_white_metrics_balanced = eval_bias(balanced_race_model, white_caucasian_group, multi_jaccard)
race_black_metrics_balanced = eval_bias(balanced_race_model, black_aa_group, multi_jaccard)

balanced_race = {
     "Race: White/Caucasian": race_white_metrics_balanced.numpy().squeeze(),
    "Race: Black/AA": race_black_metrics_balanced.numpy().squeeze(),
}
balanced_race_res = pd.DataFrame(balanced_race).T
balanced_race_res["Average"] = balanced_race_res.mean(axis=1)
balanced_race_res.columns = [val for key, val in classes.items()] + ["Average"]

balanced_race_res

In [ ]:
gender_male_metrics_stratified = eval_bias(stratified_gender_model, male_group, multi_jaccard)
gender_female_metrics_stratified = eval_bias(stratified_gender_model, female_group, multi_jaccard)

strat_gender = {
    "Gender: Male": gender_male_metrics_stratified.numpy().squeeze(),
    "Gender: Female": gender_female_metrics_stratified.numpy().squeeze(),
}
strat_gender_res = pd.DataFrame(strat_gender).T
strat_gender_res["Average"] = strat_gender_res.mean(axis=1)
strat_gender_res.columns = [val for key, val in classes.items()] + ["Average"]
strat_gender_res

In [ ]:
race_white_metrics_stratified = eval_bias(stratified_race_model, white_caucasian_group, multi_jaccard)
race_black_metrics_stratified= eval_bias(stratified_race_model, black_aa_group, multi_jaccard)

strat_race = {
    "Race: White/Caucasian": race_white_metrics_stratified.numpy().squeeze(),
    "Race: Black/AA": race_black_metrics_stratified.numpy().squeeze(),
}
strat_race_res = pd.DataFrame(strat_race).T
strat_race_res["Average"] = strat_race_res.mean(axis=1)
strat_race_res.columns = [val for key, val in classes.items()] + ["Average"]
strat_race_res

In [ ]:
gender_male_metrics = eval_bias(baseline_male_model, male_group, multi_jaccard)

gender_male = {
    "Gender: Male": gender_male_metrics.numpy().squeeze(),
}
gender_male = pd.DataFrame(gender_male).T
gender_male["Average"] = gender_male.mean(axis=1)
gender_male.columns = [val for key, val in classes.items()] + ["Average"]
gender_male

In [ ]:
gender_female_metrics = eval_bias(baseline_female_model, female_group, multi_jaccard)

gender_female = {
    "Gender: Female": gender_female_metrics.numpy().squeeze(),
}
gender_female = pd.DataFrame(gender_female).T
gender_female["Average"] = gender_female.mean(axis=1)
gender_female.columns = [val for key, val in classes.items()] + ["Average"]
gender_female

In [ ]:
race_white_metrics = eval_bias(baseline_white_model, white_caucasian_group, multi_jaccard)

race_white = {
    "Race: White/Caucasian": race_white_metrics.numpy().squeeze(),
}
race_white = pd.DataFrame(race_white).T
race_white["Average"] = race_white.mean(axis=1)
race_white.columns = [val for key, val in classes.items()] + ["Average"]
race_white

In [ ]:
race_black_metrics = eval_bias(baseline_black_model, black_aa_group, multi_jaccard)

race_black = {
    "Race: Black/AA": race_black_metrics.numpy().squeeze(),
}
race_black = pd.DataFrame(race_black).T
race_black["Average"] = race_black.mean(axis=1)
race_black.columns = [val for key, val in classes.items()] + ["Average"]
race_black

In [ ]:
def calc_fairness_score(data):
    sd = []
    ser = []

    for i, row in data.iterrows():
        sd.append(row[1:].std())
        min_group = row[1:].min()
        max_group = row[1:].max()
        ser.append((1 - min_group)/(1 - max_group))

    return sd, ser

In [ ]:
fairness_gender = {
    "Model": ["Baseline", "Balanced", "Stratified", "Group-Specific"],
    "Male": [baseline_res.loc["Gender: Male", "Average"],
             balanced_gender_res.loc["Gender: Male", "Average"],
             strat_gender_res.loc["Gender: Male", "Average"],
             gender_male.loc["Gender: Male", "Average"]],
    "Female":[baseline_res.loc["Gender: Female", "Average"],
             balanced_gender_res.loc["Gender: Female", "Average"],
             strat_gender_res.loc["Gender: Female", "Average"],
             gender_female.loc["Gender: Female", "Average"]]
}

fairness_df_gender = pd.DataFrame(fairness_gender)
sd, ser = calc_fairness_score(fairness_df_gender)
fairness_df_gender["SD"] = sd
fairness_df_gender["SER"] = ser

fairness_df_gender

In [ ]:
fairness_race = {
    "Model": ["Baseline", "Balanced", "Stratified", "Group-Specific"],
    "White/Caucasian": [baseline_res.loc["Race: White/Caucasian", "Average"],
             balanced_race_res.loc["Race: White/Caucasian", "Average"],
             strat_race_res.loc["Race: White/Caucasian", "Average"],
             race_white.loc["Race: White/Caucasian", "Average"]],
    "Black/African American":[baseline_res.loc["Race: Black/AA", "Average"],
             balanced_race_res.loc["Race: Black/AA", "Average"],
             strat_race_res.loc["Race: Black/AA", "Average"],
             race_black.loc["Race: Black/AA", "Average"]]
}

fairness_df_race = pd.DataFrame(fairness_race)
sd, ser = calc_fairness_score(fairness_df_race)
fairness_df_race["SD"] = sd
fairness_df_race["SER"] = ser

fairness_df_race